# TicTacToe

Primeramente podemos definir un jugador de TicTacToe usando un interfaz básica, dado que todo que todo juagador de TicTacToe realiza una jugada escogida de acuerdo a cierta estrategia, y es precisamente esta estrategia la única diferencia entre los distintos tipos de jugador. Siendo la estrategia más básica escoger uan jugada aleatoria dentro de todas las posibles jugadas.

In [ ]:

EMPTY = ' '
PLAYER1 = 'X'
PLAYER2 = 'O'

WIN = 1
LOSE = -1
DRAW = 0
PLAYING = 2


class Player:

    """
    This method performs a move, selected by some strategy, in a game of tic tac toe ,represented by a 3x3 matrix,
    as the player represented by 'idp'.
     
    """
    def move(self, board, idp):
        move, _  = self.best_move(board, idp)
        board[move[0]][move[1]] = idp
        return self.game_over(board, idp)

    """
    This method selects a move from the set of all available moves according to some strategy, the simplest one just select a random
    move from all available moves
    """
    def best_move(self, board, idp):
        empty_cells = set()
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY:
                    empty_cells.add((i,j))
        return empty_cells.pop(), None
    
    """
    This method checks if the given game of tic tac toe represented by a 3x3 matrix is over.
    In positive case returns the result for the player represented by 'idp'
    """
    @staticmethod
    def game_over(board, idp): 
        
        empty = 0
        ldiag = board[0][0]; rdiag = board[0][2]
        
        for i in range(3):
            if board[i][i] != ldiag: 
                ldiag = -1
            if board[i][2 - i] != rdiag:
                rdiag = -1

            row = board[i][0]; col = board[0][i]
            for j in range(3):
                if board[i][j] != row:
                    row = -1
                if board[j][i] != col:
                    col = -1
                if board[i][j] == EMPTY:
                    empty += 1

            if row != -1 and row != EMPTY:
                return WIN if row == idp else LOSE

            if col != -1 and col != EMPTY:
                return WIN if col == idp else LOSE
            
        if ldiag != -1 and ldiag != EMPTY:
            return WIN if ldiag == idp else LOSE

        if rdiag != -1 and rdiag != EMPTY:
            return WIN if rdiag == idp else LOSE
        
        return PLAYING if empty else DRAW

Entonces trantando de escoger una mejor jugada resulta obvio que es necesario poder comparar las jugadas entre ellas y por tanto una forma de calificar una jugada.
Primero notemos que calificar que tan buena es una jugada es equivalente a calificar que tan bueno es el tablero que se general al jugarla.

Calificar tableros finales es sencillo:

$$
\begin{equation}
f(b) = \begin{cases}
    1 &&  \text{, si es tablero ganador}\\
    0 && \text{, si es tablero de empate}\\
    -1 && \text{, si es tablero perdedor}\\
    \end{cases}
\end{equation}
$$

Si es un tablero intermedio podemos definir su valor recursivamente dado que su valor seria el valor del tablero final luego de una serie de jugadas óptimas a partir de dicho tablero.

Esto nos da una simple estrategia para determinar la mejor jugada:
- Dado un tablero iteramos por todas las casillas vacías
    - Jugamos en dicha casilla, si el tablero que se forma es un tablero final su resultado es uno de los tres casos base posibles, de lo contrario computamos recursivamente el valor de la mejor jugada que puede obtener nuestro oponente a partir de dicho tablero
    - Dado que si ganamos nuestro oponente pierde,si perdemos nuestro oponente gana y las tablas son iguales, entonces el valor de nuestra jugada es el opuesto de la mejor jugada de nuestro oponente
- Vamos computando iterativamente el máximo valor y jugamos en la casilla de valor máximo

Nótese que al obtener una jugada ganadora pues pudiesemos romper el ciclo ya que no habría jugada mejor, sin embargo, para hacer del comportamiento de la IA menos mecánico cada vez que se encuentra una jugada con valor igual al máximo actual se escoge aleatoriamente entre ambas jugadas.

In [ ]:
"""
This player plays optimally every time by computing the value of all possible moves every time
"""
class MinimaxPlayer(Player):

    def best_move(self, board, idp):
        move = None; score = LOSE
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY: # for each possible play
                    board[i][j] = idp
                    result = self.game_over(board, idp) # if it is a final board its result is easy to compute
                    if result != PLAYING:
                        if result >= score:    
                            move  = (i, j)
                            score = result
                    else: # hte board is an intermediate state so we compute the best play of our opponent and our score is the opposite of that
                        _, result = self.best_move(board, PLAYER1 if idp == PLAYER2 else PLAYER2) 
                        if -result > score: 
                            move = (i, j)
                            score = -result
                        if -result == score and np.random.rand(1) < 0.5:
                            move = (i, j)
                            score = -result
                    board[i][j] = EMPTY
        return move, score


A pesar de ser muy preciso este enfoque es muy lento, en una partida puede calcular alrededor de $3^9 + 3^7 + 3^5 + 3^3 + 1$ jugadas , para solventar esto utilizaremos memorización y cada vez que computemos la mejor jugada para un tablero guardaremos dicha jugada, luego cuando veamos una tablero antes de computar la mejor jugada comprobamos si lo hemos analizado antes en cuyo caso ya se sabe que jugada hacer. Esto obviamente es menos eficiente en términos de memoria pero dado que los sistemas de cómputo actuales tienen mucha memoria esto no debería representar un problema.

Para hacer esto podemos usar un arbol que represente los estados del tablero o convertir el tablero en un string y hashearlo

In [ ]:
"""
This player stores the best possible move for each board previously analyzed to improve time efficiency
"""
class MemoMinimaxPlayer(MinimaxPlayer):
    def __init__(self):
        self.moves = {}
    
    def best_move(self, board, idp):
        board_str = self.board_to_string(board)
        try:
            return self.moves[board_str]
        except KeyError:
            self.moves[board_str] = MinimaxPlayer.best_move(self, board, idp)
            return self.moves[board_str]

    def board_to_string(self, board):
        result = ""
        for i in range(3):
            for j in range(3):
                result += str(board[i][j])
        return result

Todavía podemos tratar de resolver este problema de otra forma y es evaluar el tablero de acuerdo a su estado, para este ejemplo decidí caracterizar un tablero por:
- $x_1$ número de filas/columnas/diagonales con dos de mis marcas y una casilla vacía (esto sería que estoy amenazando con ganar - es bueno y su coeficiente debería ser positivo)
- $x_2$ número de filas/columnas/diagonales con dos marcas del oponente y una casilla vacía (el oponente me amenaza - esto es malo y su coeficiente debería ser negativo)
- $x_3$ número de filas/columnas/diagonales con una de mis marcas y dos casillas vacías (esto sería que puedo amenazar)
- $x_4$ número de filas/columnas/diagonales con una marca del oponente y dos casillas vacías 
- $x_5$ es un tablero ganador
- $x_6$ es un tablero perdedor

Planteamos que $c_0 + c_1x_1 + c_2x_2 + c_3x_3 + c_4x_4+c_5x_5 +c_6x_6 = f(b)$, luego a la hora de escoger una jugada simplemente calculamos esto para cada tablero resultante posible y nos quedamos con el mejor.

Para determinar los coeficientes empezaremos coeficientes aleatorios y los iremos ajustando usando la regla LMS para actualizar los pesos utilizando como valor de entrenamiento la evaluación del tablero de nuestro siguiente turno.

In [ ]:

class TrainedPlayer(Player):

    def __init__(self, train=False):
        self.weights = np.random.rand(7)
        if train:
            self.train(100000)

    def best_move(self, board, idp):
        move = None; score = float('-inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY:
                    board[i][j] = idp
                    approx = np.dot(self.weights, self.board_features(board, idp)) # approximate f(b)
                    if approx > score:
                        move = (i, j)
                        score = approx
                    if approx == score and np.random.rand(1) < 0.5:
                        move = (i, j)
                        score = approx
                    board[i][j] = EMPTY
        return move, score

    def train(self, iters, opponent=None):
        alpha = 0.001
        for i in range(iters):
            board = [[EMPTY for j in range(3)] for i in range(3)]
            current = PLAYER1 if i % 2 == 0 else PLAYER2 # lets rotate who starts

            pre_features = self.board_features(board, current) # current board
            pre_approx = np.dot(self.weights, pre_features) # actual approximation

            move = self.move(board, current)
            board[move[0]][move[1]] = current
            result = Player.game_over(board, current)

            while result == PLAYING:
                current = PLAYER1 if current == PLAYER2 else PLAYER2
                if current == PLAYER1:
                    succ_features = self.board_features(board, current) # next board
                    succ_approx = np.dot(self.weights, self.board_features(board, current)) # next approximation
                    
                    move = self.move(board, current)
                    board[move[0]][move[1]] = current
                    result = Player.game_over(board, current)
                    
                    self.update_weights(alpha, succ_approx, pre_approx, pre_features) # update the weights
                    pre_features = succ_features # udpdate current board
                    pre_approx = np.dot(self.weights, self.board_features(board, current)) # update current approx with new weights
                else:
                    move = self.move(board, current)
                    board[move[0]][move[1]] = current
                    result = Player.game_over(board, current)

            if result == WIN:
                succ_approx = 1 if current == (PLAYER1 if i % 2 == 0 else PLAYER2) else -1
            elif result == LOSE:
                succ_approx = -1 if current == (PLAYER1 if i % 2 == 0 else PLAYER2) else 1
            else:
                succ_approx = 0
            
            self.update_weights(alpha, succ_approx, pre_approx, pre_features)
                    

    def update_weights(self, learning_rate, train_v, approx, features): # weight update LMS rule
        for i in range(len(self.weights)):
            self.weights[i] = self.weights[i] + learning_rate * (train_v - approx) * features[i]


    def board_features(self, board, idp):
        x0 = 1  # Constant
        x1 = 0  # Number of rows/columns/diagonals with two of our own pieces and one emtpy field
        x2 = 0  # Number of rows/columns/diagonals with two of opponent's pieces and one empty field
        x3 = 0  # Number of rows/columns/diagonals with one own piece and two empty fields
        x4 = 0  # Number of rows/columns/diagonals with one opponent's piece and two empty fields
        x5 = 0  # Number of rows/columns/diagonals with three own pieces
        x6 = 0  # Number of rows/columns/diagonals with three opponent's pieces

        for i in range(3):
            own_rows = 0
            own_columns = 0
            enemy_rows = 0
            enemy_columns = 0
            empty_rows = 0
            empty_columns = 0
            for j in range(3):
                if board[i][j] == EMPTY:
                    empty_rows += 1
                elif board[i][j] == idp:
                    own_rows += 1
                else:
                    enemy_rows += 1
                if board[j][i] == 0:
                    empty_columns += 1
                elif board[j][i] == idp:
                    own_columns += 1
                else:
                    enemy_columns += 1

            if own_rows == 2 and empty_rows == 1:
                x1 += 1
            if enemy_rows == 2 and empty_rows == 1:
                x2 += 1
            
            if own_columns == 2 and empty_columns == 1:
                x1 += 1
            if enemy_columns == 2 and empty_columns == 1:
                x2 += 1

            if own_rows == 1 and empty_rows == 2:
                x3 += 1
            if own_columns == 1 and empty_columns == 2:
                x3 += 1
            
            if enemy_rows == 1 and empty_rows == 2:
                x4 += 1
            if enemy_columns == 1 and empty_columns == 2:
                x4 += 1

            if own_rows == 3:
                x5 += 1
            if own_columns == 3:
                x5 += 1

            if enemy_rows == 3:
                x6 += 1
            if enemy_columns == 3:
                x6 += 1

        for i in range(2):
            own_diagonal = 0
            enemy_diagonal = 0
            empty_diagonal = 0
            for j in range(3):
                if i == 0:
                    diagonal = board[2-j][j]
                else:
                    diagonal = board[j][j]
                if diagonal == EMPTY:
                    empty_diagonal += 1
                elif diagonal == idp:
                    own_diagonal += 1
                else:
                    enemy_diagonal += 1
            if own_diagonal == 2 and empty_diagonal == 1:
                x1 += 1
            if enemy_diagonal == 2 and empty_diagonal == 1:
                x2 += 1
            if own_diagonal == 1 and empty_diagonal == 2:
                x3 += 1
            if enemy_diagonal == 1 and empty_diagonal == 2:
                x4 += 1
            if own_diagonal == 3:
                x5 += 1
            if enemy_diagonal == 3:
                x6 += 1


        return [x0, x1, x2, x3, x4, x5, x6]
        


Si bien esta solución debería ser más eficiente en terminos de espacio que el Minimax con memorización (solo almacena un array de coeficientes) también es más imprecisa, en los resultados actuales el jugador resulta demasiado ofensivo por lo que si el primero en jugar es un jugador óptimo siempre pierde al no intentar detener las amenazas del contrario y tratar de amenzar él.